# US airline sentiment analysis using tweets data
dataset link: https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment.

The data shows whether the sentiment of the tweets was positive, neutral, or negative for six US airlines

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use(style='seaborn')
%matplotlib inline
from sklearn.model_selection import train_test_split
from nltk import ngrams
from nltk.corpus import stopwords
import nltk
from collections import defaultdict
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import string
string.punctuation
import warnings
warnings.filterwarnings("ignore")
from numpy import hstack
import tensorflow
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import ReLU
from keras.layers import LeakyReLU
from keras.layers import Softmax
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SimpleRNN, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.utils import np_utils
import tensorflow.keras as keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
#from pydub import AudioSegment
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#!pip uninstall nltk==3.2.5 -y
#!pip install nltk==3.6.2
# make sure nltk version is 3.6.2. if not uncomment above and run 
import nltk
print('The nltk version is {}.'.format(nltk.__version__))


The nltk version is 3.6.2.


In [4]:
df = pd.read_csv('Tweets.csv')

In [5]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [6]:
df['text']

0                      @VirginAmerica What @dhepburn said.
1        @VirginAmerica plus you've added commercials t...
2        @VirginAmerica I didn't today... Must mean I n...
3        @VirginAmerica it's really aggressive to blast...
4        @VirginAmerica and it's a really big bad thing...
                               ...                        
14635    @AmericanAir thank you we got on a different f...
14636    @AmericanAir leaving over 20 minutes Late Flig...
14637    @AmericanAir Please bring American Airlines to...
14638    @AmericanAir you have my money, you change my ...
14639    @AmericanAir we have 8 ppl so we need 2 know h...
Name: text, Length: 14640, dtype: object

# Clean the data.
remove stopwords, tags, twitter handles etc

In [7]:
#Preprocessing ---Removing stopwords and performing Stemming/Lemmitization
stop_words = set(stopwords.words('english'))
# Remove stopwords and other tags
# old dataset required deep preprocessing
def remove_stopwords(tweets):
    all_words = ""
    for word in word_tokenize(tweets):
        if not(word.lower() in stop_words):
            word = word.lower()
            all_words+= (word + " ")
    return all_words
  

def remove_tags(string): # remove punctutation and symbols etc
    removelist = ""
    result = re.sub(r'http\S+', '', string) 
    result = re.sub('@[\w]+','',result) 
    result = re.sub(r"(?<!#)\d+", "", result)  
    result = re.sub(r'[^a-zA-Z]', ' ', result)
    result = re.sub('\s*\\b([a-z]|[a-z]{1})\\b', ' ', result)
    result = re.sub(r'[\W_ + ½]+', ' ', result)
    result = result.lower()
    return result

In [8]:
## apply above functions
df['text']=df['text'].apply(lambda x : remove_tags(x))
#remove stopwords 
df['text']= df['text'].apply(lambda x:remove_stopwords(x))


In [9]:
df['text']

0                                                    said 
1                 plus added commercials experience tacky 
2                  today must mean need take another trip 
3        really aggressive blast obnoxious entertainmen...
4                                    really big bad thing 
                               ...                        
14635                  thank got different flight chicago 
14636    leaving minutes late flight warnings communica...
14637           please bring american airlines blackberry 
14638    money change flight answer phones suggestions ...
14639    ppl need know many seats next flight plz put u...
Name: text, Length: 14640, dtype: object

# Build vocab

In [10]:
vocab = []
for tweet in df['text']:
    vocab.extend((tweet.split()))
vocab = list(set(vocab))  # get rid of repeated words


# tokenizing and Sequencing for the NN input

In [11]:
max_features = 1000
token = Tokenizer(num_words=max_features, split = ' ')
token.fit_on_texts(df['text'].values)

X = token.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

In [12]:
import pickle
pickle.dump(token,open('token_.pkl','wb'))

# model the NN


In [12]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(196, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 128)           128000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 18, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 3)                 591       
                                                                 
Total params: 383,391
Trainable params: 383,391
Non-trainable params: 0
_________________________________________________________________


# Convert labels

In [13]:
# encoding for labels
from sklearn.preprocessing import LabelEncoder
labels = df['airline_sentiment']
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

from keras.utils import np_utils
import tensorflow.keras as keras
num_classes = 3
labels = keras.utils.to_categorical(encoded_labels, num_classes)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,labels, test_size=0.1, random_state=1)

In [15]:
batch_size = 64
history = model.fit(X_train, y_train, epochs=5, batch_size=batch_size, verbose=1)

Epoch 1/5
206/206 [==============================] - 24s 105ms/step - loss: 0.7190 - accuracy: 0.7003
Epoch 2/5
206/206 [==============================] - 21s 100ms/step - loss: 0.5392 - accuracy: 0.7811
Epoch 3/5
206/206 [==============================] - 23s 109ms/step - loss: 0.5016 - accuracy: 0.7967
Epoch 4/5
206/206 [==============================] - 22s 107ms/step - loss: 0.4789 - accuracy: 0.8067
Epoch 5/5
206/206 [==============================] - 23s 113ms/step - loss: 0.4654 - accuracy: 0.8111


In [17]:
model.evaluate(X_test, y_test, batch_size=batch_size, verbose=2)

23/23 - 0s - loss: 0.5641 - accuracy: 0.7712 - 420ms/epoch - 18ms/step


[0.5640700459480286, 0.7711748480796814]

# test random comments

In [18]:
text = ['I would only suggest if you have no other option']
text = token.texts_to_sequences(text)
text = pad_sequences(text, maxlen=18, dtype='int32', value=0)
res = model.predict(text, batch_size=1,verbose = 1)
if np.argmax(res) == 0:print("Negative Comment")
elif np.argmax(res) == 1:print("Neutral Comment")
else: print("Postitive Comment")


1/1 [==============================] - 0s 276ms/step
Negative Comment


# Save model for flask deployment

In [21]:
model.save('model_sentiment')

INFO:tensorflow:Assets written to: model_sentiment\assets
